In [ ]:
! pip install -r requirements.txt

In [2]:
import gymnasium as gym
import os
import matplotlib.pyplot as plt
import json # for dumping debug data
import time # for benchmarking 
from ddqn_tfkeras import SingleQAgent, DoubleQAgent
import numpy as np
import random

LEARN_EVERY = 4
def train_agent(atype='double', n_episodes=2000, load_latest_model=False):
    print("Training a {} DQN TF-Keras agent on {} episodes. Pretrained model = {}".format(atype,n_episodes,load_latest_model))
    env = gym.make("LunarLander-v2")
    if atype == 'double':
        agent = DoubleQAgent(gamma=0.99, epsilon=1.0, epsilon_dec=0.995, lr=0.001, mem_size=2000000, batch_size=128, epsilon_end=0.01)
    elif atype == 'single':
        agent = SingleQAgent(gamma=0.99, epsilon=0.01, epsilon_dec=0.996, lr=0.001, mem_size=2000000, batch_size=128, epsilon_end=0.01)
    
    if load_latest_model:
        agent.load_saved_model('{}_dqn_tfk_model.h5'.format(atype))
        print('Loaded most recent {} model.'.format(atype))
        
    scores = []
    eps_history = []
    start = time.time()
    for i in range(n_episodes):
        terminated = False
        truncated = False
        score = 0
        
        steps = 0
        state = env.reset()[0]
        while not (terminated or truncated):
            action = agent.choose_action(state)
            new_state, reward, terminated, truncated, info = env.step(action)
            agent.save(state, action, reward, new_state, terminated)
            state = new_state
            if steps > 0 and steps % LEARN_EVERY == 0:
                agent.learn()
            steps += 1
            score += reward
        eps_history.append(agent.epsilon)
        scores.append(score)
        avg_score = np.mean(scores[max(0, i-100):(i+1)])

        if (i+1) % 5 == 0 and i > 0:
            # Report expected time to finish the training
            print('Episode {} in {:.2f} min. Expected total time for {} episodes: {:.0f} min. [{:.2f}/{:.2f}]'.format((i+1), 
                                                                                                                      (time.time() - start)/60, 
                                                                                                                      n_episodes, 
                                                                                                                      (((time.time() - start)/i)*n_episodes)/60, 
                                                                                                                      score, 
                                                                                                                      avg_score))

        if (i+1) % 25 == 0 and i > 0:
            # Save the model every N-th step just in case
            agent.save_model('{}_dqn_tfk_model.h5'.format(atype))
            with open("{}_dqn_tfk_model_scores_{}.json".format(atype,int(time.time())), "w") as fp:
                json.dump(scores, fp)
            with open("{}_dqn_tfk_model_history_{}.json".format(atype, int(time.time())), "w") as fp:
                json.dump(eps_history, fp)

    return agent
    
train_agent(atype='single', n_episodes=76, load_latest_model=True)

Training a single DQN TF-Keras agent on 76 episodes. Pretrained model = True
Loaded most recent single model.
Episode 5 in 4.00 min. Expected total time for 76 episodes: 76 min. [222.46/122.26]
Episode 10 in 6.10 min. Expected total time for 76 episodes: 52 min. [247.00/186.92]
Episode 15 in 8.50 min. Expected total time for 76 episodes: 46 min. [231.59/212.37]
Episode 20 in 10.46 min. Expected total time for 76 episodes: 42 min. [271.60/224.07]
Episode 25 in 13.23 min. Expected total time for 76 episodes: 42 min. [171.44/210.52]
Episode 30 in 15.68 min. Expected total time for 76 episodes: 41 min. [265.88/212.14]
Episode 35 in 17.72 min. Expected total time for 76 episodes: 40 min. [280.70/221.10]
Episode 40 in 19.39 min. Expected total time for 76 episodes: 38 min. [238.50/223.58]
Episode 45 in 22.49 min. Expected total time for 76 episodes: 39 min. [237.61/221.84]
Episode 50 in 24.84 min. Expected total time for 76 episodes: 39 min. [285.07/225.42]
Episode 55 in 29.10 min. Expected 

In [3]:
# # Visualize the model
# import gymnasium as gym
# import os
# import matplotlib.pyplot as plt
# os.environ["SDL_VIDEODRIVER"] = "dummy"
# from IPython.display import clear_output

# def visualize_model(name, atype='single'):
#     env = gym.make("LunarLander-v2", render_mode="rgb_array")
#     agent = SingleQAgent(gamma=0.99, epsilon=0.0, lr=0.0005, mem_size=1000000, batch_size=64, epsilon_end=0.01)
#     agent.load_saved_model(name)
#     state, info = env.reset(seed=42)
#     for _ in range(5):
#         terminated = False
#         truncated = False
#         while not (terminated or truncated):
#             action = agent.choose_action(state)
#             new_state, reward, terminated, truncated, info = env.step(action)
#             if truncated:
#                 print("Truncated game at {}", steps)
#             state = new_state
#             clear_output(wait=True)
#             plt.imshow( env.render() )
#             plt.show()
#         state = env.reset()[0]
#     env.close()

# visualize_model('double_dqn_model_epoch_50_fc256xfc256.h5', atype='double')

KeyboardInterrupt: 